In [5]:
# load all libraries
import pandas as pd
import ncbi.datasets

In [6]:
# Start a Datasets gene API instance
api_client = ncbi.datasets.ApiClient()
ds_gene_instance = ncbi.datasets.GeneApi(api_client)

# Retrieve gene summaries for the three genes using NCBI Gene IDs
gene_summary = ds_gene_instance.gene_metadata_by_id([1903612])

# Look up the symbols, chromosome number and SwissProt accession for each gene ,84347,140724,140725,140726,140727,140728,140729,501675,688285
def report_on_gene_descriptors(gene_summary, leader='\t', report_errors=True):
    if report_errors:
        for message in gene_summary.messages or []:
            print(f'{leader}Error for: ({",".join(message.error.invalid_identifiers)})')
            print(f'{leader}{leader}Reason: ({message.error.reason})')

    if not gene_summary.genes:
        print(f'{leader}No genes found')
        return

    for gene in map(lambda g: g.gene, gene_summary.genes):
        print(f'{gene.symbol} (GeneID: {gene.gene_id}), Chromosome: {gene.chromosomes}, SwissProt: {gene.swiss_prot_accessions}')

report_on_gene_descriptors(gene_summary)

AttributeError: 'NoneType' object has no attribute 'symbol'

In [7]:
import pandas as pd

def get_acc(species):
    acc = []
    df = pd.read_excel("C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\All_species.xlsx", sheet_name=species)
    for i in df["accession_number"]:
        acc.append(i)
    return acc




In [8]:
#search location by gene id and protein accession number

acc = ["NP_001139808.1","NP_001191139.1"]
geneID = [283953,283999]

def searchlocation(acc, geneID):
    api_client = ncbi.datasets.ApiClient()
    ds_gene_instance = ncbi.datasets.GeneApi(api_client)
    gene_summary = ds_gene_instance.gene_metadata_by_id([geneID])

    for i in gene_summary.genes:
        chromosome = (i.gene.chromosomes[0])
        complement = i.gene.orientation
        symbol = i.gene.symbol
        cdna = ""
        start = ""
        stop = ""
        complement = ""
        accession_version = ""
        cdnastart = ""
        cdnastop = ""
        exon = []
        exonstart = 0
        
        if not i.gene.transcripts:
            pass
        else:
            for l in i.gene.transcripts:
                if not l.protein:
                    pass
                elif l.protein.accession_version == acc:
                    accession_version = i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.accession_version
                    cdna = i.gene.transcripts[i.gene.transcripts.index(l)].accession_version


                    cdnastart = (i.gene.transcripts[i.gene.transcripts.index(l)].cds.range[0].begin)
                    cdnastop = (i.gene.transcripts[i.gene.transcripts.index(l)].cds.range[0].end)


                    start = (i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.range[0]['begin'])
                    stop = (i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.range[0]['end'])
                    complement = (i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.range[0]['orientation'])

                    exon = [len(i.gene.transcripts[i.gene.transcripts.index(l)].genomic_locations[0].exons)]
                    for num in range (len(i.gene.transcripts[i.gene.transcripts.index(l)].genomic_locations[0].exons)):
                        exon.append(i.gene.transcripts[i.gene.transcripts.index(l)].genomic_locations[0].exons[num])
                    exonstart = int(exon[1]['end'])-int(exon[1]['begin'])-int(cdnastart)
                    tempexon = int(exonstart)
                    print(tempexon)

                else:
                    pass

        if str(complement)=="minus":
            location = symbol + "," + acc + ",\"Chromosome: " + chromosome + "; " + accession_version + "(" + start + ".." + stop + ", complement)\"," + cdna + ", " + cdnastart + ", " + cdnastop + ", " + str(exonstart)
            for num in range (2,len(exon)-1):
                tempexon += int(exon[num]['end'])-int(exon[num]['begin'])
                location += "," + str(tempexon)
        elif str(complement) != "minus":
            location = symbol +  "," + acc + ",Chromosome: " + chromosome + "; " + accession_version + "(" + start + ".." + stop + ")," + cdna + "," + cdnastart + ", " + cdnastop + ", " + str(exonstart)
            for num in range (2,len(exon)-1):
                tempexon += int(exon[num]['end'])-int(exon[num]['begin'])
                location += "," + str(tempexon)
        return location

for i in range(len(acc)):
    print(searchlocation(acc[i], geneID[i]))


218
TMEM114,NP_001139808.1,"Chromosome: 16; NC_000016.10(8569500..8590511, complement)",NM_001146336.2, 674, 1345, 218,298,435
-594
TMEM235,NP_001191139.1,Chromosome: 17; NC_000017.11(78231293..78240987),NM_001204210.2,642, 1313, -594,187,267,404,653


In [9]:
def writecsvtxt(acc,geneID):
    temp = []
    for i in range(len(acc)):
        temp.append(searchlocation(acc[i], geneID[i]))
    return temp


txt = writecsvtxt(acc,geneID)

for i in txt:
    print(i)

218
-594
TMEM114,NP_001139808.1,"Chromosome: 16; NC_000016.10(8569500..8590511, complement)",NM_001146336.2, 674, 1345, 218,298,435
TMEM235,NP_001191139.1,Chromosome: 17; NC_000017.11(78231293..78240987),NM_001204210.2,642, 1313, -594,187,267,404,653


In [10]:
### output to csv
def outputcsv(txt):
    file = open("output.csv",'w')
    file.write("protein,accession,location,cDNAs,begin,end,exon_boundary1,exon_boundary2,exon_boundary3,exon_boundary4,exon_boundary5,exon_boundary6,exon_boundary7,exon_boundary8\n")
    for i in range(len(txt)):
        if i != len(txt)-1:
            file.write(txt[i] + "\n")
        else:
            file.write(txt[i])
    file.close()
outputcsv(txt)

In [11]:
### get begin and end
def get_start_end():
    df = pd.read_csv("output.csv")
    return list(df['begin']), list(df['end']),list(df['cDNAs'])

start = get_start_end()[0]
stop = get_start_end()[1]
cDNAs = get_start_end()[2]

In [12]:
####get sequence

import re
import pandas as pd
from urllib.request import urlopen
import os


def AccesiontoID(accession_num):
    url = "https://www.ncbi.nlm.nih.gov/protein/"+str(accession_num)+"?report=gilist&log$=seqview&format=text"
    df = urlopen(url)
    for lines in df:
        lines = lines.decode("utf-8")
        if "<pre>" in lines:
            id = lines[5:].rstrip()
    return id

def get_sequence(geneid,start,stop):
    temp = ""
    url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=fasta&id=" + geneid + "&from=" + str(start) + "&to=" + str(stop) + "&extrafeat=null&conwithfeat=on&hide-cdd=on"
    file = urlopen(url)
    for line in file:
        temp += line.decode("utf-8")
    return temp

for i in range(len(cDNAs)):
    txt = get_sequence(AccesiontoID(cDNAs[i]),start[i],stop[i])
    print(txt)

>NM_001146336.2:674-1345 Homo sapiens transmembrane protein 114 (TMEM114), transcript variant 1, mRNA
ATGCGGGTGCACCTGGGCGGGCTGGCCGGCGCGGCTGCGCTGACCGGGGCGCTCAGCTTTGTGCTCCTGG
CGGCCGCCATCGGCACGGACTTCTGGTATATCATTGACACCGAGCGGCTGGAGAGGACTGGCCCGGGGGC
GCAGGACCTGCTGGGGTCCATCAATCGCAGCCAGCCCGAGCCTCTGAGCTCCCACTCCGGCCTCTGGCGG
ACCTGCCGGGTGCAGAGCCCGTGCACACCGCTGATGAACCCCTTCAGGCTGGAGAACGTGACAGTCAGCG
AATCGAGCCGGCAACTTCTCACCATGCATGGGACATTTGTGATTCTGCTGCCGCTCAGCCTGATCCTGAT
GGTTTTTGGGGGGATGACGGGGTTTCTGAGCTTCCTCCTCCAAGCCTACCTCCTCCTCCTGCTCACTGGA
ATTCTCTTCCTCTTTGGAGCCATGGTGACCCTCGCTGGGATCAGCGTCTACATAGCGTATTCAGCCGCCG
CCTTCCGGGAGGCGCTGTGTCTCTTGGAGGAGAAGGCCCTCCTGGACCAGGTGGACATCAGCTTCGGCTG
GTCCCTGGCCCTGGGCTGGATCAGCTTCATCGCCGAGCTGCTCACCGGGGCAGCCTTCCTGGCAGCAGCC
CGCGAGCTCAGCCTGAGACGGAGGCAGGACCAGGCCATATGA


>NM_001204210.2:642-1313 Homo sapiens transmembrane protein 235 (TMEM235), transcript variant 1, mRNA
ATGGCCCGGCTGGGCGCGCTGCTCCTGGCCGCCGCCCTGGGTGCACTGCTCAGCTTCGCGCTCCTGGCCG
CCGCGGTCGCCAGCGACTACTGGTACATCCTGGAGGTGGCG

In [13]:
path = "C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\"

def createfilename(txt):
    path = "C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\"
    pattern = "(?<=\\s).+"
    temp = re.findall(pattern,txt)
    temp = re.sub("[;:]","",temp[0])
    return path + temp + ".txt"

def writetxt(path,txt):
    with open (path,"w") as obj:
        obj.write(txt)


temp = ""
for i in range(len(cDNAs)):
    txt = get_sequence(AccesiontoID(cDNAs[i]),start[i],stop[i])
    temp+=txt
    writetxt(createfilename(txt),txt)

writetxt("C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\" + "all.txt",temp)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\Homo sapiens transmembrane protein 114 (TMEM114), transcript variant 1, mRNA.txt'

In [14]:
gene_summary

{'genes': [{'query': ['1903612'],
            'warnings': [{'gene_warning_code': 'UNRECOGNIZED_GENE_ID',
                          'message': 'The gene you requested, (1903612) is not '
                                     'a recognized NCBI Gene ID.\n',
                          'reason': 'This GeneID is unrecognized.',
                          'unrecognized_identifier': '1903612'}]}]}

In [15]:
#search location by gene id and protein accession number

acc = ["XP_003456412.1","XP_005463339.1","XP_005463337.1"]
geneID = [100698262,100696764,100696764]

for i in range(len(acc)):
    print(searchlocation(acc[i], geneID[i]))

-62
cacng5b,XP_003456412.1,Chromosome: LG4; NC_031969.2(18973098..18995016),XM_003456364.4,421, 1251, -62,193,279,419,564
-47
cacng5,XP_005463339.1,Chromosome: LG6; NC_031971.2(28425321..28443748),XM_005463282.3,299, 1132, -47,193,279,419,564
-47
cacng5,XP_005463337.1,Chromosome: LG6; NC_031971.2(28425320..28443748),XM_005463280.3,300, 1175, -47,193,234,320,460,605


# Gene structure and transmembrane domains analysis

In [16]:
# read data from excel

import pandas as pd

dataf = pd.read_excel("data\\transmembrane\\Book1.xlsx", sheet_name = "Sheet3")
textname = []
nuclen = []
aalen = []
exonboun = []
trans = []
templist = []

for i in range(0,len(dataf["species"])):
    textname.append(dataf["species"][i]+" "+dataf["protein"][i])
    templen = int(dataf["end"][i])-int(dataf["begin"][i])+1
    nuclen.append(templen)
    aalen.append(templen/3)
    for j in range(1,6):
        templist.append(dataf["exon_boundary"+str(j)][i])
    templist = [x for x in templist if x ==x ]
    exonboun.append(templist)
    templist = []
    for k in range(1,6):
        templist.append(dataf["trans"+str(k)+"_start"][i])
        templist.append(dataf["trans"+str(k)+"_end"][i])
    templist = [x for x in templist if x ==x ]
    trans.append(templist)
    templist = []


In [17]:
# create figure

from PIL import Image, ImageDraw, ImageFont

newImage = Image.new('RGBA', (2000,2500), 'White')
drawObj = ImageDraw.Draw(newImage)

drawstart = 450
recx1 = 5
recx2 = 15
recy1 = 20
recy2 = 40
texty = 15
triy1 = 50
triy2 = triy1+30
gap = 180
adjust = 1.5

fontInfo = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf',45)

for item in range(0,len(dataf)):

    drawObj.text((20,texty), textname[item], fill='Black', font=fontInfo)

    drawObj.rectangle((drawstart,recy1,drawstart+adjust*aalen[item],recy2),fill='lightgrey')

    for i in range (0,len(trans[item]),2):
        drawObj.rectangle((drawstart+round(adjust*trans[item][i]),recx1,drawstart+round(adjust*trans[item][i+1]),recx2),fill='Black')
    
    for i in range (0,len(exonboun[item])):
        drawObj.polygon([(450+(round(exonboun[item][i]/nuclen[item]*adjust*aalen[item]))-10,triy2),(450+(round(exonboun[item][i]/nuclen[item]*adjust*aalen[item])),triy1),(450+(round(exonboun[item][i]/nuclen[item]*adjust*aalen[item]))+10,triy2)],fill='Black')

    texty += gap
    recy1 += gap
    recy2 += gap
    recx1 += gap
    recx2 += gap
    triy1 += gap
    triy2 += gap


drawObj.polygon([(950-10,texty+35),(950,texty+5),(950+10,texty+35)],fill='Black')
drawObj.text((1000,texty), ": Exon boundaries", fill='Black', font=fontInfo, align='right')

drawObj.rectangle((925,texty+90,970,texty+100),fill='Black')
drawObj.text((1000,texty+70), ": Predicted transmembrane segments", fill='Black', font=fontInfo, align='right')

newImage.save("structureout3.png")